<a href="https://colab.research.google.com/github/Subina00/blockchain-project/blob/main/Copy_of_Assessment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import hashlib #For SHA-256 cryptographic hash
import time # For timestamping blocks
from typing import List # For type hinting


# ------------------------- CUSTOM EXCEPTIONS -------------------------
class BlockchainError(Exception):
    """Base class for Blockchain exceptions."""
    pass


class InvalidBlockError(BlockchainError):
    """Raised on failure of validation check of a single block."""
    pass


# ------------------------- BLOCK CLASS -------------------------
class Block:
    """
    Represents one single block of blockchain.
    Stores transaction data, timestamp, proof-of-work, and cryptographic hash.
    """

    def __init__(self, index: int, previous_hash: str, timestamp: float, data: str, proof: int) -> None:
        self.index = index #Index is the basis of the block in the chain (0 for genesis)
        self.previous_hash = previous_hash #It stores the hash of the former block (enforces chain connection).
        self.timestamp = timestamp #Stores in what time the block was made
        self.data = data #The data contained in the block as transactions or options.
        self.proof = proof #Proof-of-work (nonce) used for mining.
        self.hash = self.hash_calculation() #Calculate the unique hash of the block by the content of the block

    def hash_calculation (self) -> str:
        """
        Calculates the block (using attributes) with the hash of SHA-256.
        The result is a 64-character hexadecimal string (base 16).
        """
        #Create all attributes as one string.
        value_to_hash = f"{self.index}{self.previous_hash}{self.timestamp}{self.data}{self.proof}"
        #Hash and encode the string by SHA-25
        return hashlib.sha256(value_to_hash.encode()).hexdigest()

    def to_dict(self) -> dict:
        """ Transforms the block to a dictionary to facilitate export or display."""
        return {
            "index": self.index,
            "previous_hash": self.previous_hash,
            "timestamp": self.timestamp,
            "data": self.data,
            "proof": self.proof,
            "hash": self.hash
        }


# ---------------------- BLOCKCHAIN CLASS -----------------------
class Blockchain:
    """
    Handles the blockchain and gives the means of validation, mining and addition of new blocks.
    """

    def __init__(self, difficulty: int = 4) -> None:
        #Blockchain begins with the genesis block
        self.chain: List[Block] = [self.create_genesis_block()]
        #The difficulty sets the number of leading zeros needed in the hash
        self.difficulty = difficulty

    def create_genesis_block(self) -> Block:
        """Generates the initial block (genesis block) that has fixed data."""
        return Block(0, "0", time.time(), "Genesis Block", 0)

    def get_latest_block(self) -> Block:
        """Returns the last block that was inserted."""
        return self.chain[-1]

    def proof_of_work(self, block: Block) -> Block:
        """
        Uses mining to discover a proof (nonce) so that the hash value of a block has a specified number of leading zeros (as per the difficulty).
        """
        block.proof = 0 # Start nonce at 0
        block.hash = block.hash_calculation()

        #Increment the proof next to the hash until the difficulty condition is met.
        while not block.hash.startswith('0' * self.difficulty):
            block.proof += 1
            block.hash = block.hash_calculation()
        return block

    def add_block(self, new_block: Block) -> None:
        """
        Verifies the validity of a block by adding one to the blockchain.
        Raises:
            InvalidBlockError: In the case of block linkage or the hash not being valid.
        """
        #Make the last block in the chain equal to the previous hash
        new_block.previous_hash = self.get_latest_block().hash
        # Mining the block (get valid hash)
        mined_block = self.proof_of_work(new_block)

        #Before adding, validate the mined block
        if not self.is_valid_new_block(mined_block, self.get_latest_block()):
            raise InvalidBlockError(f"Block {new_block.index} is invalid!")
        self.chain.append(mined_block)

    def add_data(self, data: str) -> None:
        """ Makes a new block of the given data and mines it."""
        new_block = Block(
            index=len(self.chain),#Block index is latest length of chain
            previous_hash=self.get_latest_block().hash,
            timestamp=time.time(), #current timestamp
            data=data,
            proof=0
        )
        self.add_block(new_block)

    def is_valid_new_block(self, current: Block, previous: Block) -> bool:
        """Verifies validity of new block in relation to the previous block."""
        if current.previous_hash != previous.hash:
            return False
        if current.hash != current.hash_calculation():
            return False
        if not current.hash.startswith('0' * self.difficulty):
            return False
        return True

    def is_chain_valid(self) -> bool:
        """Checks the integrity of entire blockchain."""
        for i in range(1, len(self.chain)):
            if not self.is_valid_new_block(self.chain[i], self.chain[i - 1]):
                print(f"[ERROR] Blockchain is invalid at block. {self.chain[i].index}.")
                return False
        return True


# ----------------------- CLI INTERFACE ------------------------

def display_chain(blockchain: Blockchain) -> None:
    """Displays the blockchain contents."""
    for block in blockchain.chain:
        print(f"\n--- Block {block.index} ---")
        print(f"Timestamp: {time.ctime(block.timestamp)}") #Conversion of timestamp to human datetime format
        print(f"Data: {block.data}")
        print(f"Hash: {block.hash}")
        print(f"Previous Hash: {block.previous_hash}")
        print(f"Proof: {block.proof}")
        print("-" * 50)


def menu():
    """Basic CLI for the blockchain."""
    blockchain = Blockchain()
    while True:
      # Display menu options
        print("\n_____Blockchain Menu______")
        print("1. Mine Data (Add Block)")
        print("2. Display Blockchain")
        print("3. Check Blockchain Validity")
        print("4. Exit")
        choice = input("Enter choice: ").strip()

        if choice == '1':
            data = input("Please input data for mining:  ")
            print(" Mining block...")
            blockchain.add_data(data)
            print(" New block mined and added to the blockchain!")
        elif choice == '2':
            display_chain(blockchain)
        elif choice == '3':
            print("Blockchain Validity:", blockchain.is_chain_valid())
        elif choice == '4':
            print(" Exiting...")
            break
        else:
            print("[ERROR] No valid choice. Try again.")


if __name__ == "__main__":
    menu()



_____Blockchain Menu______
1. Mine Data (Add Block)
2. Display Blockchain
3. Check Blockchain Validity
4. Exit
Enter choice: 1
Please input data for mining:  dfvbghh
 Mining block...
 New block mined and added to the blockchain!

_____Blockchain Menu______
1. Mine Data (Add Block)
2. Display Blockchain
3. Check Blockchain Validity
4. Exit
Enter choice: 2

--- Block 0 ---
Timestamp: Tue Jul 22 04:42:43 2025
Data: Genesis Block
Hash: f6fd1ba548b59d989d72054c259c70d9855c60f5ec758a617ddfc36716a3310f
Previous Hash: 0
Proof: 0
--------------------------------------------------

--- Block 1 ---
Timestamp: Tue Jul 22 04:42:50 2025
Data: dfvbghh
Hash: 00000df8f2e61445a8421ccb433703a9f4308632572c17b58944554299c69c49
Previous Hash: f6fd1ba548b59d989d72054c259c70d9855c60f5ec758a617ddfc36716a3310f
Proof: 131423
--------------------------------------------------

_____Blockchain Menu______
1. Mine Data (Add Block)
2. Display Blockchain
3. Check Blockchain Validity
4. Exit
